In [1]:
using DelimitedFiles
using JLD2

using RCall

#randomly select 50 cells
using StatsBase
using Random


In [2]:
ko = load_object("./KOSimulatedDataSets.jld2")
# rn = load_object("./randomNetworkTimeSeriesSimData.jld2")

10×801×2000×117 Array{Float32, 4}:
[:, :, 1, 1] =
 0.01  0.201194    0.366262    …  0.0231889   0.0057749   0.00833542
 0.01  0.208994    0.406957       0.0124905   0.0130699   0.00937771
 0.01  0.00598702  0.00780563     0.102561    0.0531797   0.0531797
 0.01  0.00792303  0.00222643     0.0178097   0.00811467  0.011533
 0.01  0.0149566   0.00338211     2.08355     2.05938     2.07396
 0.01  0.0156799   0.0132413   …  0.0221611   0.0365067   0.0399525
 0.01  0.01        0.00572541     1.40909     1.48211     1.53062
 0.01  0.00717269  0.00387277     2.08978     2.12016     2.23406
 0.01  0.107738    0.178732       0.00462356  0.00462356  0.00262957
 1.0   1.12734     0.926824       0.0         0.0         0.0

[:, :, 2, 1] =
 0.01  0.198371   0.404999    0.54122      …  0.00731777   0.0113064
 0.01  0.206182   0.371077    0.624473        0.001567     0.001567
 0.01  0.01       0.00168218  0.00526659      0.000656675  0.00335998
 0.01  0.0220946  0.0339415   0.0321026       0.00920262 

In [3]:

biData = ko[4:end,2:end,:,1]

#UMAP in R to determine the branch points
Random.seed!(123)
cellsToUse = sample(1:2000, 50, replace = false)
biData1 = Matrix{Float32}(undef, 7, 40000)
function createMat(cellsToUse, biData)
    j =1
    for i in cellsToUse
        biData1[:,1+(j-1)*800:(800*j)] = biData[:,:,i]
        j+=1
    end
    return biData1
end
biData1 = createMat(cellsToUse, biData)

@rput biData1

### WARNING behavior of scaledata changed, have to use normalizedata

R"""
library(Seurat)

colnames(biData1) = c(1:dim(biData1)[2])
rownames(biData1) = c(1:dim(biData1)[1])
seu = CreateSeuratObject(biData1)

seu@meta.data$time = rep(c(1:800), 50)
seu <- NormalizeData(seu)
seu <- ScaleData(seu)
seu <- RunPCA(seu, verbose = FALSE, npcs = 5, features = c(1:dim(biData1)[1]))
seu <- FindNeighbors(seu, dims = 1:3)
seu <- FindClusters(seu)
seu <- RunUMAP(seu, dims = 1:3, verbose = FALSE)
saveRDS(seu, "bifurcatingDataSeu1.rds")
pdf("bifurcatingSimUMAP.pdf")
p <- FeaturePlot(seu, features = 'time', cols = rev(heat.colors(6)))
print(p)
dev.off()
"""


┌ Warning: RCall.jl: Loading required package: SeuratObject
│ Loading required package: sp
│ 
│ Attaching package: ‘SeuratObject’
│ 
│ The following objects are masked from ‘package:base’:
│ 
│     intersect, t
│ 
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Warning: Data is of class matrix. Coercing to dgCMatrix.
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Normalizing layer: counts
│ Performing log-normalization
│ 0%   10   20   30   40   50   60   70   80   90   100%
│ [----|----|----|----|----|----|----|----|----|----|
│ **************************************************|
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Centering and scaling data matrix
  |======================================================================| 100%
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Warning in irlba(A = t(x = object), nv = npcs, ...) :
│   You're computing too large a percen

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 40000
Number of edges: 1019380

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8982
Number of communities: 23
Elapsed time: 2 seconds


┌ Warning: RCall.jl: 0%   10   20   30   40   50   60   70   80   90   100%
│ [----|----|----|----|----|----|----|----|----|----|
│ **************************************************|
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Warning: The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
│ To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
│ This message will be shown once per session
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172


RObject{IntSxp}
null device 
          1 
